# Import

In [ ]:
import pandas
import sklearn
import evaluation
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
GPU_fix = "MemoryFix" # Choose "Disable", "MemoryFix" or "None"

if GPU_fix == "Disable":
    try:
        # Disable all GPUS
        tf.config.set_visible_devices([], 'GPU')
        visible_devices = tf.config.get_visible_devices()
        for device in visible_devices:
            assert device.device_type != 'GPU'
    except:
        # Invalid device or cannot modify virtual devices once initialized.
        pass
elif GPU_fix == "MemoryFix":
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

# Read training data

In [94]:
# https://www.kaggle.com/sionek/ugbc-gs
folder = 'tau_data/'
data = pandas.read_csv(folder + 'training.csv.zip', index_col='id')
#--------------- feature engineering -------------- #
def add_features(df):
    # features used by the others on Kaggle
    df['NEW_FD_SUMP']=df['FlightDistance']/(df['p0_p']+df['p1_p']+df['p2_p'])
    df['NEW5_lt']=df['LifeTime']*(df['p0_IP']+df['p1_IP']+df['p2_IP'])/3
    df['p_track_Chi2Dof_MAX'] = df.loc[:, ['p0_track_Chi2Dof', 'p1_track_Chi2Dof', 'p2_track_Chi2Dof']].max(axis=1)
    #df['flight_dist_sig'] = df['FlightDistance']/df['FlightDistanceError'] # modified to:
    df['flight_dist_sig2'] = (df['FlightDistance']/df['FlightDistanceError'])**2
    # features from phunter
    df['flight_dist_sig'] = df['FlightDistance']/df['FlightDistanceError']
    df['NEW_IP_dira'] = df['IP']*df['dira']
    df['p0p2_ip_ratio']=df['IP']/df['IP_p0p2']
    df['p1p2_ip_ratio']=df['IP']/df['IP_p1p2']
    df['DCA_MAX'] = df.loc[:, ['DOCAone', 'DOCAtwo', 'DOCAthree']].max(axis=1)
    df['iso_bdt_min'] = df.loc[:, ['p0_IsoBDT', 'p1_IsoBDT', 'p2_IsoBDT']].min(axis=1)
    df['iso_min'] = df.loc[:, ['isolationa', 'isolationb', 'isolationc','isolationd', 'isolatione', 'isolationf']].min(axis=1)
    # My:
    # new combined features just to minimize their number;
    # their physical sense doesn't matter
    df['NEW_iso_abc'] = df['isolationa']*df['isolationb']*df['isolationc']
    df['NEW_iso_def'] = df['isolationd']*df['isolatione']*df['isolationf']
    df['NEW_pN_IP'] = df['p0_IP']+df['p1_IP']+df['p2_IP']
    df['NEW_pN_p']  = df['p0_p']+df['p1_p']+df['p2_p']
    df['NEW_IP_pNpN'] = df['IP_p0p2']*df['IP_p1p2']
    df['NEW_pN_IPSig'] = df['p0_IPSig']+df['p1_IPSig']+df['p2_IPSig']
    #My:
    # "super" feature changing the result from 0.988641 to 0.991099
    df['NEW_FD_LT']=df['FlightDistance']/df['LifeTime']
    return df

print("Add features")
train_df = add_features(data)

print("Eliminate features")
filter_out = ['id', 'min_ANNmuon', 'production', 'mass', 'signal',
              'SPDhits','CDF1', 'CDF2', 'CDF3',
              'p0_pt', 'p1_pt', 'p2_pt',
              'p0_p', 'p1_p', 'p2_p', 'p0_eta', 'p1_eta', 'p2_eta',
              'isolationa', 'isolationb', 'isolationc', 'isolationd', 'isolatione', 'isolationf',
              'p0_IsoBDT', 'p1_IsoBDT', 'p2_IsoBDT',
              'p0_IP', 'p1_IP', 'p2_IP',
              'IP_p0p2', 'IP_p1p2',
              'p0_track_Chi2Dof', 'p1_track_Chi2Dof', 'p2_track_Chi2Dof',
              'p0_IPSig', 'p1_IPSig', 'p2_IPSig',
              'DOCAone', 'DOCAtwo', 'DOCAthree']

Add features
Eliminate features


In [ ]:
import featuretools as ft
data = pandas.read_csv(folder + 'training.csv.zip')
es = ft.EntitySet(id='data')
es.entity_from_dataframe(entity_id = 'df', dataframe = data, index = 'id')
print(es)

In [ ]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'df', 
max_depth = 1, 
verbose = 1, 
n_jobs = 3,
trans_primitives = ['add_numeric', 'multiply_numeric'])

In [ ]:
feature_matrix.columns

In [ ]:
def preprocess_data(X, scaler=None):
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    X = scaler.transform(X)
    return X

In [95]:
#modified
variables_mod = list(f for f in data.columns if f not in filter_out)
print(len(variables_mod))

28


In [96]:
print(len([i for i in train_df['signal'].to_numpy() if i == 0]))
train_df = data[variables_mod]
#train_df = preprocess_data(train_df)

25879


In [97]:
msk = np.random.rand(len(train_df)) < 0.8
train = train_df[msk]
val = train_df[~msk]

In [98]:
print('validation samples:',len(val))
print('train samples:',len(train))

validation samples: 13645
train samples: 53908


# Baseline training

In [99]:
#Create training arrays
x_train = train
y_train = data['signal'][msk].to_numpy()
y_train = np.expand_dims(y_train,1)
x_val = val
y_val = data['signal'][~msk].to_numpy()
y_val = np.expand_dims(y_val,1)

In [ ]:
check_agreement = pandas.read_csv(folder + 'check_agreement.csv.zip', index_col='id')
check_agreement = add_features(check_agreement)
check_agreement1 = check_agreement[variables_mod].to_numpy()


In [ ]:
check_correlation = pandas.read_csv(folder + 'check_correlation.csv.zip', index_col='id')
check_correlation = add_features(check_correlation)
check_correlation1 = check_correlation[variables_mod].to_numpy()


In [132]:

def my_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()
    agreement_probs = model1.predict(check_agreement1).squeeze()
    ks = evaluation.compute_ks(
        agreement_probs[check_agreement['signal'].values == 0],
        agreement_probs[check_agreement['signal'].values == 1],
        check_agreement[check_agreement['signal'] == 0]['weight'].values,
        check_agreement[check_agreement['signal'] == 1]['weight'].values)
    correlation_probs = model1.predict(check_correlation1).squeeze()
    cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])
    return bce(y_true, y_pred) + exp(ks) + 10*exp(cvm)

In [134]:
import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    #self.inputs = tf.keras.layers.InputLayer(len(variables_mod))
    self.bnorm1 = tf.keras.layers.BatchNormalization()
    self.bnorm2 = tf.keras.layers.BatchNormalization()
    self.bnorm3 = tf.keras.layers.BatchNormalization()
    self.bnorm4 = tf.keras.layers.BatchNormalization()
    self.dropout1 = tf.keras.layers.Dropout(0.3)
    self.dropout2 = tf.keras.layers.Dropout(0.3)
    self.dropout3 = tf.keras.layers.Dropout(0.3)
    self.dropout4 = tf.keras.layers.Dropout(0.3)
    self.dense1 = tf.keras.layers.Dense(200, activation=tf.keras.layers.LeakyReLU(alpha=0.2))
    self.dense2 = tf.keras.layers.Dense(200, activation=tf.keras.layers.LeakyReLU(alpha=0.2))
    self.dense3 = tf.keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(alpha=0.2))
    self.dense4 = tf.keras.layers.Dense(50, activation=tf.keras.layers.LeakyReLU(alpha=0.2))
    self.dense5 = tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)

  def call(self, inputs):
    #x = self.inputs(inputs)
    x = self.bnorm1(inputs)
    x = self.dropout1(x)
    x = self.dense1(x)
    x = self.bnorm2(x)
    x = self.dropout2(x)
    x = self.dense2(x)
    x = self.bnorm3(x)
    x = self.dropout3(x)
    x = self.dense3(x)
    x = self.bnorm4(x)
    x = self.dropout4(x)
    x = self.dense4(x)
    bce = tf.keras.losses.BinaryCrossentropy()
    agreement_probs = self.predict(check_agreement1).squeeze()
    ks = evaluation.compute_ks(
        agreement_probs[check_agreement['signal'].values == 0],
        agreement_probs[check_agreement['signal'].values == 1],
        check_agreement[check_agreement['signal'] == 0]['weight'].values,
        check_agreement[check_agreement['signal'] == 1]['weight'].values)
    correlation_probs = self.predict(check_correlation1).squeeze()
    cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])
    loss = bce(y_true, y_pred) + exp(ks) + 10*exp(cvm)
    self.add_loss(self)
    return self.dense5(x)

model = MyModel()

model.compile( optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])
#model.build(input_shape = (len(variables_mod),1))
#model.summary()


Подключение LR Scheduler

In [143]:
def scheduler(epoch, lr):
    if epoch % 300 != 0:
        return lr
    else:
        return lr/5

Создание тюнера

In [ ]:
from kerastuner.tuners import RandomSearch
def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=len(variables_mod)))
    model.add(tf.keras.layers.BatchNormalization())
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=10,
                                            max_value=200,
                                            step=10),
                               activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
        model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='BinaryCrossentropy',
        metrics=['accuracy'])
    return model


tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=2,
    directory='tuner1',
    project_name='ttt')

In [ ]:
tuner.search(x_train, y_train,
             epochs=100,
             validation_data=(x_val, y_val), batch_size = 128)

In [ ]:
tuner.results_summary()

In [ ]:
model_tuned = tuner.get_best_models(num_models=1)[0]

In [144]:
model1 = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(len(variables_mod)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(200, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(200, activation=tf.keras.layers.LeakyReLU(alpha=0.2)), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(alpha=0.2)), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(50, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),  
  tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)
])
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input
from tensorflow.keras.layers import PReLU
model = Sequential()
model.add(Input(shape=len(variables_mod)))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dropout(0.15))
model.add(Dense(75))
model.add(PReLU())

model.add(Dropout(0.13))
model.add(Dense(60))
model.add(PReLU())

model.add(Dropout(0.11))
model.add(Dense(45))
model.add(PReLU())

model.add(Dropout(0.09))
model.add(Dense(30))
model.add(PReLU())

model.add(Dropout(0.07))
model.add(Dense(15))
model.add(PReLU())

model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='BinaryCrossentropy', optimizer='rmsprop',metrics=['accuracy'])

In [100]:
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train, random_state=0)

In [ ]:
history_1 = model1.fit(x_train, y_train, epochs = 1000, batch_size = 128, validation_data = (x_val,y_val))

Epoch 1/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.4158 - accuracy: 0.8164 - val_loss: 0.3465 - val_accuracy: 0.8518
Epoch 2/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3892 - accuracy: 0.8286 - val_loss: 0.3377 - val_accuracy: 0.8554
Epoch 3/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3846 - accuracy: 0.8325 - val_loss: 0.3385 - val_accuracy: 0.8538
Epoch 4/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3795 - accuracy: 0.8334 - val_loss: 0.3346 - val_accuracy: 0.8578
Epoch 5/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3782 - accuracy: 0.8339 - val_loss: 0.3329 - val_accuracy: 0.8557
Epoch 6/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3769 - accuracy: 0.8353 - val_loss: 0.3290 - val_accuracy: 0.8587
Epoch 7/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3743 - accuracy: 0.8377 - val_loss: 0.3294 - val_accuracy:

422/422 [==============================] - 2s 4ms/step - loss: 0.3598 - accuracy: 0.8431 - val_loss: 0.3268 - val_accuracy: 0.8568
Epoch 58/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3597 - accuracy: 0.8441 - val_loss: 0.3256 - val_accuracy: 0.8641
Epoch 59/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3599 - accuracy: 0.8451 - val_loss: 0.3275 - val_accuracy: 0.8637
Epoch 60/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3605 - accuracy: 0.8429 - val_loss: 0.3289 - val_accuracy: 0.8607
Epoch 61/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3584 - accuracy: 0.8452 - val_loss: 0.3275 - val_accuracy: 0.8621
Epoch 62/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3598 - accuracy: 0.8441 - val_loss: 0.3274 - val_accuracy: 0.8593
Epoch 63/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3614 - accuracy: 0.8424 - val_loss: 0.3243 - val_accuracy: 0.8638

422/422 [==============================] - 2s 4ms/step - loss: 0.3566 - accuracy: 0.8459 - val_loss: 0.3264 - val_accuracy: 0.8621
Epoch 114/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3573 - accuracy: 0.8454 - val_loss: 0.3285 - val_accuracy: 0.8624
Epoch 115/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3589 - accuracy: 0.8440 - val_loss: 0.3252 - val_accuracy: 0.8609
Epoch 116/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3565 - accuracy: 0.8454 - val_loss: 0.3240 - val_accuracy: 0.8622
Epoch 117/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3589 - accuracy: 0.8442 - val_loss: 0.3226 - val_accuracy: 0.8622
Epoch 118/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3589 - accuracy: 0.8438 - val_loss: 0.3285 - val_accuracy: 0.8600
Epoch 119/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3573 - accuracy: 0.8453 - val_loss: 0.3249 - val_accuracy: 

Epoch 169/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3545 - accuracy: 0.8462 - val_loss: 0.3239 - val_accuracy: 0.8635
Epoch 170/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3576 - accuracy: 0.8462 - val_loss: 0.3242 - val_accuracy: 0.8632
Epoch 171/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3576 - accuracy: 0.8466 - val_loss: 0.3250 - val_accuracy: 0.8630
Epoch 172/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3574 - accuracy: 0.8455 - val_loss: 0.3246 - val_accuracy: 0.8627
Epoch 173/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3556 - accuracy: 0.8476 - val_loss: 0.3249 - val_accuracy: 0.8625
Epoch 174/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3556 - accuracy: 0.8460 - val_loss: 0.3243 - val_accuracy: 0.8638
Epoch 175/1000
422/422 [==============================] - 2s 4ms/step - loss: 0.3560 - accuracy: 0.8447 - val_loss: 0.3241 -

422/422 [==============================] - 2s 5ms/step - loss: 0.3565 - accuracy: 0.8471 - val_loss: 0.3252 - val_accuracy: 0.8629
Epoch 226/1000
422/422 [==============================] - 2s 5ms/step - loss: 0.3547 - accuracy: 0.8452 - val_loss: 0.3238 - val_accuracy: 0.8617
Epoch 227/1000
422/422 [==============================] - 2s 5ms/step - loss: 0.3545 - accuracy: 0.8470 - val_loss: 0.3270 - val_accuracy: 0.8623
Epoch 228/1000
422/422 [==============================] - 2s 5ms/step - loss: 0.3558 - accuracy: 0.8457 - val_loss: 0.3241 - val_accuracy: 0.8627
Epoch 229/1000
422/422 [==============================] - 2s 5ms/step - loss: 0.3540 - accuracy: 0.8467 - val_loss: 0.3229 - val_accuracy: 0.8635
Epoch 230/1000
422/422 [==============================] - 2s 5ms/step - loss: 0.3538 - accuracy: 0.8463 - val_loss: 0.3209 - val_accuracy: 0.8630
Epoch 231/1000
422/422 [==============================] - 2s 5ms/step - loss: 0.3567 - accuracy: 0.8456 - val_loss: 0.3265 - val_accuracy: 

In [ ]:

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model1.fit(x_train, y_train, epochs = 1000, batch_size = 128, validation_data = (x_val,y_val), callbacks = [callback])

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
model_tuned = tuner.get_best_models(num_models=1)[0]
history2 = model_tuned.fit(x_train, y_train, epochs = 100, batch_size = 128, validation_data = (x_val,y_val), callbacks = [callback])

In [142]:

x_train.shape
check_agreement1.shape

(331147, 28)

In [ ]:
model1.optimizer.lr.numpy()

In [ ]:
model1.save_weights('./models/my_model/m_bnorm_mod')

In [ ]:
model1.save_weights('./models/my_model/m_bnorm')
model.save_weights('./models/imported_model/i_bnorm')

# Check agreement test

In [ ]:
import matplotlib.pylab as pylab

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': '150',
         'axes.titlesize':'40',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

plt.figure(figsize=(16,10))

plt.xlabel('Epochs',size=30)
plt.ylabel("Loss",size=30)
val = plt.plot(history.epoch, history.history['loss'],
             label='Train loss', linewidth = 2)
plt.plot(history.epoch, history.history['val_'+'loss'],
             label='Validation loss', linewidth = 2)
plt.legend(prop={'size':30})
plt.xticks(fontsize = 25)
plt.xticks(np.arange(0, max(history.epoch)+10, 500.0))
plt.yticks(fontsize = 25)
plt.xlim([0,max(history.epoch)+1])
plt.show()
plt.savefig('test.png', bbox_inches='tight')

In [ ]:
model_tuned.predict(train_df)

In [119]:
check_agreement = pandas.read_csv(folder + 'check_agreement.csv.zip', index_col='id')
check_agreement = add_features(check_agreement)
check_agreement1 = check_agreement[variables_mod].to_numpy()
agreement_probs = model.predict(check_agreement1).squeeze()
print(agreement_probs)
ks = evaluation.compute_ks(
    agreement_probs[check_agreement['signal'].values == 0],
    agreement_probs[check_agreement['signal'].values == 1],
    check_agreement[check_agreement['signal'] == 0]['weight'].values,
    check_agreement[check_agreement['signal'] == 1]['weight'].values)
print('KS metric', ks, ks < 0.09)

[1.8562473e-15 1.8866131e-33 8.4488590e-11 ... 0.0000000e+00 7.7070257e-17
 1.4698114e-18]
KS metric 0.19992033859579014 False


# Check correlation test

In [120]:
check_correlation = pandas.read_csv(folder + 'check_correlation.csv.zip', index_col='id')
check_correlation = add_features(check_correlation)
check_correlation1 = check_correlation[variables_mod].to_numpy()
correlation_probs = model.predict(check_correlation1).squeeze()
print(correlation_probs)
cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])
print('CvM metric', cvm, cvm < 0.002)

[1.0725059e-01 0.0000000e+00 2.8520667e-12 ... 7.0625579e-01 9.3361038e-01
 9.0962332e-01]
CvM metric 0.09743354571968595 False


# Compute weighted AUC on the training data with min_ANNmuon > 0.4

In [ ]:
train_eval = data[data['min_ANNmuon'] > 0.4]
train_probs = model_tuned.predict(train_eval[variables_mod].to_numpy()).squeeze()
AUC = sklearn.metrics.roc_auc_score(train_eval['signal'], train_probs)
print('AUC', AUC)

# Predict test, create file for kaggle

In [ ]:
result = pandas.DataFrame({'id': test.index})
result['prediction'] = model.predict(test[variables].to_numpy()).squeeze()

In [ ]:
result.to_csv('submission.csv', index=False, sep=',')

In [ ]:
!ls -l submission.csv

In [ ]:
result.to_csv?